In [35]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import re
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")

In [3]:
movies.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
movies = movies.merge(credits, on="title")


In [6]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [7]:
# taking these columns
# genres
# id
# keywords
# overview
# tagline
# title
# cast
# crew


In [8]:
movies = movies[["genres",
"id",
"keywords",
"overview",
"title",
"cast",
"crew"]]

In [9]:
movies.head()

,genres,id,keywords,overview,title,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...",Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca...",John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [10]:
movies.isnull().sum()

genres      0
id          0
keywords    0
overview    3
title       0
cast        0
crew        0
dtype: int64

In [11]:
movies.dropna(inplace = True)

In [12]:
movies.duplicated().sum()


np.int64(0)

In [13]:
movies.iloc[0, 0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [14]:
# fn for handling genres and keywords
def convert(obj):
    l=[]
    for data in ast.literal_eval(obj):
        l.append(data["name"])
    return l
        

In [15]:
movies["genres"] = movies["genres"].apply(convert)

In [16]:
movies["keywords"] = movies["keywords"].apply(convert)

In [17]:
# fn for getting top 3 casts
def convert3(obj):
    l=[]
    count = 0
    for data in ast.literal_eval(obj):
        if count != 3:
            l.append(data["name"])
            count += 1
        else:
            break
    return l

In [18]:
movies["cast"] = movies["cast"].apply(convert3)

In [19]:
movies["crew"] = movies["crew"].apply(lambda x : [data["name"] for data in  ast.literal_eval(x) if data["job"] == "Director"])

In [20]:
# creating list of overciew column
movies["overview"] = movies["overview"].apply(lambda x : x.split())

In [21]:
# removing extra spaces because because model can confuse bw sam worthington and sam mendes

movies["keywords"] = movies["keywords"].apply(lambda x : [data.replace(" ", "") for data in x ]) 
movies["genres"] = movies["genres"].apply(lambda x : [data.replace(" ", "") for data in x ]) 
movies["cast"] = movies["cast"].apply(lambda x : [data.replace(" ", "") for data in x ]) 
movies["crew"] = movies["crew"].apply(lambda x : [data.replace(" ", "") for data in x ]) 


In [22]:
# merging all necessary columns to one
movies["tags"] = movies["keywords"] + movies["genres"] + movies["cast"] + movies["crew"] + movies["overview"]

In [23]:
# creating a new df from the required columns
new_df = movies[["id", "title", "tags"]].copy()

In [24]:
# converting tags into a str
new_df["tags"] = new_df["tags"].apply(lambda x : " ".join(x))

In [25]:
new_df["tags"] = new_df["tags"].apply(lambda x : x.lower())

In [26]:
# correcting indexes
new_df = new_df.reset_index(drop=True)


In [27]:
# using stemming to handle morphological variation in CV's vocab
# ex = play, playing, played but root word in play

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    ls = []
    for word in text.split():
        ls.append(ps.stem(word))
    return (" ").join(ls)

new_df["tags"] = new_df["tags"].apply(stem)

    




In [28]:
# implementing bag of words to create vectors of tags
cv = CountVectorizer(max_features = 5000, stop_words="english")
vectors = cv.fit_transform(new_df["tags"]).toarray()

In [29]:
print(vectors.shape)

(4806, 5000)


In [30]:
similarity = cosine_similarity(vectors)
similarity.shape

(4806, 4806)

In [31]:
def recommend(movie):
    movie_idx = new_df[new_df["title"] == movie].index[0]
    distances = similarity[movie_idx]
    top_movies_idxs = np.argsort(distances)[::-1][1:6]
    return new_df["title"].iloc[top_movies_idxs]

In [32]:
recommend("Avatar")

2405                         Aliens
539                      Titan A.E.
507                Independence Day
1214    Aliens vs Predator: Requiem
1202                      Predators
Name: title, dtype: object

In [33]:
recommend("Harry Potter and the Philosopher's Stone")

4448                                    1982
2429    The Adventures of Elmo in Grouchland
4529                                The R.M.
3664                               Flying By
2655                                  Khumba
Name: title, dtype: object

In [37]:
 # creating pkl of movies data to load in web app

with open("movies.pkl", "wb") as f:
    pickle.dump(new_df, f)

In [38]:
# creating pkl of similarities for web app to recommend movies

with open("similarities.pkl", "wb") as f:
    pickle.dump(similarity, f)

In [66]:
# reducing the size of of similarities to store top 20 movies
top_20_similarity = []
for distances in similarity:
    top_20_similarity.append(np.argsort(distances)[::-1][1:21])
with open("top_20_similar_movies.pkl", "wb") as f:
    pickle.dump(top_20_similarity, f)

In [68]:
new_df[new_df["id"] == 679]

,id,title,tags
2405,679,Aliens,android extraterrestrialtechnolog spacemarin s...
